In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
%matplotlib inline

import seaborn as sns
sns.set(color_codes=True)

In [20]:
import requests
import math
import time
import argparse
import threading

node = '178.238.236.94'
nthreads = 10

def blocks_reader(seq_from, seq_to, index):
    print (seq_from, seq_to)
    thread_blocks[index] = requests.get('http://%s:6869/blocks/seq/%d/%d' % (node, seq_from, seq_to)).json()

In [140]:
max_days = -1
max_assets = -1
ndays = 0
count = 0
last = requests.get('http://' + node + ':6869/blocks/height').json()['height']
prevdate = ''
asset_txs={}
block_with_first_asset = 236967

data = pd.DataFrame()
args = {}


In [145]:
try:
    
    for n in range(int(math.ceil((last - block_with_first_asset) / (nthreads * 100)) + 1)):
        print n
        thread_blocks = []
        thread=[]
        for t in range(nthreads):
            thread_blocks.append('')
            thread.append(threading.Thread(target=blocks_reader, args=(max(1, last - (n + 1) * (nthreads * 100) + t*100 + 1), last - n * (nthreads * 100) - ((nthreads * 100) - 100) + t*100, t)))
            thread[t].start()
        blocks=[]
        for t in range(nthreads):
            thread[t].join()
            blocks = blocks + thread_blocks[t]
        for block in reversed(blocks):
            txs = block['transactions']
            for tx in reversed(txs):
                date = time.strftime('%m/%d/%Y', time.gmtime(tx['timestamp']/1000.))
                if date!=prevdate:
                    if ndays == max_days:
                        raise
                    ndays +=1
                prevdate = date

                if tx['type']==3:
                    txdata = pd.DataFrame.from_dict(tx, orient='index').transpose()
                    data = data.append(txdata, ignore_index=True)
                    
#                     print (tx)
                    if count == max_assets:
                        raise
                    issue_time = time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(tx['timestamp']/1000.))
                    issuer = tx['sender']
                    assetid = tx['assetId']
                    name = tx['name'][:20].encode('ascii', 'ignore')
                    description = tx['description'][:40].encode('ascii', 'ignore').replace('\n',' ')
                    qt = tx['quantity']
                    dec = tx['decimals']
                    amount = '{:.{prec}f}'.format(qt / 10. ** dec, prec=dec)
                    if tx['assetId'] in asset_txs:
                        txcount = asset_txs[tx['assetId']]
                    else:
                        txcount = 0
                    if txcount > 0:
                        count += 1
                        print ("%6d  %-45s %-20s %24s" % (count, assetid, name, amount))
                elif tx['type']==4:
                    if tx['assetId'] in asset_txs:
                        asset_txs[tx['assetId']] += 1
                    else:
                        asset_txs[tx['assetId']] = 0

except:
    pass

0
(577036, 577135)
(577136, 577235)
(577236, 577335)
(577336, 577435)
(577436, 577535)
(577536, 577635)(577636, 577735)

(577736, 577835)
(577836, 577935)
(577936, 578035)
   821  HEG23mEwNxFTThb4LMoazzK9XgPo1ZCCQaRxbCBb6Wcg  Brouzouf                 92233720368.00000000
   822  EcwbiJCmPHeCDq14V1a4Sn6V3MqAiZKWn1vq3AaUStA5  TLP                           100000.00000000
1
(576036, 576135)
(576136, 576235)
(576236, 576335)
(576336, 576435)
(576436, 576535)
(576536, 576635)
(576636, 576735)
(576736, 576835)
(576836, 576935)
(576936, 577035)
   823  CtNnGiVmQF4BJQNsH73Gfr1VS2jPnJQTGjSf6HU2fomc  Coin Coin                  100000000.00000000
   824  AuCU7V9M1qUcBHPiJ2cjQRJUXnxDmkCQqBjGsySWXfHy  WeebCoin                     100000557.000000
   825  BvQbMQ2g52dZpZUcu2syaedqFajUFhTpb5wHc35DprZN  Sestertius                   1000000000.00000
   826  EN9Pyts1jsR7ZzG6XVCWYnztBnU9zbVB7PBKa7oJsup5  Sesterce                     1000000000.00000
   827  GuZm68fxq6dNQt8sRmt9PtsvAyAG8p3d5d23uzgutd57  Ea

In [146]:
data2 = data.set_index(['assetId'])
data2 = data

data2['timestamp'] = pd.to_datetime((data2['timestamp']),unit='ns')
data2['amount'] = data2[['quantity']].apply(lambda x: x/10. ** data2['decimals']).astype(int)
data3 = data2[['name','amount','description','reissuable','timestamp','id','sender']]

data3

ValueError: mixed datetimes and integers in passed array

In [97]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print categorical_columns
print numerical_columns

[u'reissuable', u'fee', u'sender', u'assetId', u'timestamp', u'description', u'senderPublicKey', u'decimals', u'signature', u'quantity', u'type', u'id', u'name']
[u'amount']


In [98]:
data[categorical_columns].describe()

,reissuable,fee,sender,assetId,timestamp,description,senderPublicKey,decimals,signature,quantity,type,id,name
count,53,53,53,53,53,53,53,53,53,53,53,53,53
unique,2,1,44,53,53,52,44,8,53,34,1,53,53
top,True,100000000,3PMrTCBTbF9xxvqwfCEKJEibtWSa2bXW5Lu,2TQn3H226vGeYGfJHutS5uPJpTsDUrZTKypLAKkaYZ7m,1499789218929,,6Jy2p4R3FC1FkPrbhcJ7i894aX9fyCWA16YhPaJiN1L9,8,3jUMGvPqpGRjrP8RttiHp4Q6SvyMiCmN28KkpHPBbqe1pK...,10000000000000000,3,2TQn3H226vGeYGfJHutS5uPJpTsDUrZTKypLAKkaYZ7m,VeriCoin
freq,27,53,3,1,1,2,3,24,1,6,53,1,1


In [27]:
# data[categorical_columns].hist()

In [28]:
#конвертируем текстовые значения в числовые индексы

from sklearn import preprocessing
from collections import defaultdict
d = defaultdict(preprocessing.LabelEncoder)
data_fit = data

# Encoding the variable
data_fit = data.apply(lambda x: d[x.name].fit_transform(x))

# Inverse the encoded
# fit.apply(lambda x: d[x.name].inverse_transform(x))

# Using the dictionary to label future data
# data_fit.apply(lambda x: d[x.name].transform(x))

# print d['pol'].transform('муж')
# print d['professiya'].inverse_transform(8)

data_fit[:4]


,reissuable,fee,sender,timestamp,description,senderPublicKey,decimals,signature,quantity,type,id,name
assetId,,,,,,,,,,,,
HpWM5mqidwPcdx6Df5ntuoGoGNwHyYsgYbC6UH8ZBHNY,0,0,27,34,22,26,7,34,17,0,34,13
Aak9GTmmEs4w4gAuaSvTeGfN3AkPmLMNb4E1wg3PhfM,1,0,6,33,14,6,7,16,20,0,18,24
FeA6DmyEBWHmE41mnwdiDs4e8mfrrLiMampKfLSmiJb4,0,0,21,32,11,17,1,20,8,0,30,15
6HMPr2tzJDoSKWcR7e6uSwcjgjfBGiGDB319FzGKoCgq,1,0,27,31,4,26,3,8,5,0,11,6


In [ ]:
# binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
# nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
# print binary_columns, nonbinary_columns

In [ ]:
# res = pd.get_dummies(data_x)
# res[:5]

# Тренеруем алгоритм

In [29]:
def predskazanie(data,chto=''):
    """
    Основная функция предсказаний, 
    Передаем столбцы и предсказываем значение
    """
    
    X = data.drop((chto), axis=1)  # Выбрасываем столбец 
    print data.shape

    y = data[chto]
    feature_names = X.columns
    print feature_names

    # подготавливаем выборку
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

    N_train, _ = X_train.shape 
    N_test,  _ = X_test.shape 
    print N_train, N_test

    # запускаем лес
    from sklearn import ensemble
    rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
    rf.fit(X_train, y_train)

    err_train = np.mean(y_train != rf.predict(X_train))
    err_test  = np.mean(y_test  != rf.predict(X_test))
    print 'Процент ошибки: ', err_train, err_test

    #выбираем важные
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]

    print("Важность признаков по убыванию:")
    for f, idx in enumerate(indices):
        print("{:2d}. признак '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

    #график важных
    d_first = data.shape[1]-1
    plt.figure(figsize=(8, 8))
    plt.title("Важность признаков для предсказании: " + chto)
    plt.barh(range(d_first), importances[indices[:d_first]], align='center')
    plt.yticks(range(d_first), np.array(feature_names)[indices[:d_first]])
#     plt.xlim([-1, d_first]);
    
    return X


## Зависимость

In [ ]:
zavis = predskazanie(data_fit.drop(['id'], axis=1),chto='reissuable')

# ОБЩИЙ АНАЛИЗ ДАННЫХ

In [32]:
## Профессия и Зарплата

xxx = 'professiya'
yyy = 'zarplata'

ax = sns.barplot(x=xxx, y=yyy, data=data_fit);
ax.set(xlabel='Профессии', ylabel='Зарплата')
plt.show()

maxval = data_fit.loc[data_fit[xxx].idxmax()][xxx]

for i in range(maxval+1):
    print i, d[xxx].inverse_transform(i)

ValueError: Could not interpret input 'professiya'